# 说明
这个笔记本用于Atlas 200DK摄像头采集图片数据，保存到Atlas 200DK本地文件夹。
启动后，用户可以实时查看200DK摄像头视频画面，每点击一次**保存图片**按钮，会将当前视频画面截获，并保存到本地文件夹

- 文件夹命令：Images-<系统时间> , 例如：Images-2022-11-10-13_36_54
- 图片命名：<系统时间>.jpg , 例如：2022-11-10-13_43_26.jpg

# 依赖条件
1. 下载[Ascend Sample代码仓](https://gitee.com/ascend/samples)到A200DK
2. 参考[Ascend Sample代码仓 python环境准备](https://gitee.com/ascend/samples/tree/master/python/environment)章节准备依赖环境，主要是`acllite`库的安装和环境变量设置


# 实现说明
通过4个线程（摄像头实时画面捕获线程、界面显示线程、图片文件保存线程、主线程）来完成图片数据的采集、保存。主线程用于整个程序的启动-停止控制。
其中用到了`queue.Queue`模块来缓存图片数据，使用缓存队列的目的是对实时摄像头画面截获 与 慢速图片数据写入文件 2个线程之间传递数据。


In [1]:
import os
import sys
import time
import queue
import threading

import cv2
import numpy as np
import ipywidgets as widgets
from ipyevents import Event 
from IPython.display import display

from cameracapture import CameraCapture
from acllite_resource import AclLiteResource
from acllite_imageproc import AclLiteImageProc


CAMERA_FRAME_WIDTH = 1080
CAMERA_FRAME_HEIGHT = 720

# https://pypi.org/project/ipywidgets/
imgbox = widgets.Image(format='jpg', height=CAMERA_FRAME_HEIGHT, width=CAMERA_FRAME_WIDTH)


FLAG = True
IMAGE = False

q = queue.Queue(maxsize=10)

uuid_str = time.strftime("%Y-%m-%d-%H_%M_%S", time.localtime())
tmp_folder_name ='Images-%s' % uuid_str
os.makedirs(tmp_folder_name, exist_ok=True)

# 获取摄像头画面并实时显示 线程函数
def capture_video():
    display(imgbox)
    
    #Initialize acl
    acl_resource = AclLiteResource()
    acl_resource.init()
    dvpp = AclLiteImageProc(acl_resource)

    #Open the CARAMER0 camera on the development board
    cap = CameraCapture(0)
    
    global FLAG
    while FLAG:
        # Read a picture from the camera
        image = cap.read()  # YUV420SP NV12 format
        if image is None:
            print("Get memory from camera failed")
            break
        jpeg_image = dvpp.jpege(image) # encode to jpeg format
        img = jpeg_image.byte_data_to_np_array() 
        imgbox.value = img
        
        global IMAGE
        if IMAGE:
            if q.full():
                continue
            else:
                q.put(img)
            IMAGE = False


# 捕获摄像头画面 IMAGE 控制全局变量
def cap_image(change):
    global IMAGE
    IMAGE = True

# 捕获摄像头画面按钮显示 线程函数
def button_control():
    # 创建按钮
    button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
    get_image_button = widgets.Button(description='保存图片', button_style='danger', layout=button_layout)
    
    # 显示按钮
    middle_box = widgets.HBox([get_image_button], layout=widgets.Layout(align_self='center'))
    
    # link buttons to actions
    get_image_button.on_click(cap_image) 
    display(middle_box)


# 保存图片到文件 线程函数
def save_image():
    while FLAG:
        img = q.get()
        uuid_str = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime())
        tmp_file_name ='%s.jpg' % uuid_str
        tmp_file_path = os.path.join(tmp_folder_name, tmp_file_name)
        img.tofile(tmp_file_path)
    

In [2]:
thread1 = threading.Thread(name='t1',target=capture_video, args=())
thread1.start()

thread2 = threading.Thread(name='t2',target=button_control, args=())
thread2.start()

thread3 = threading.Thread(name='t3',target=save_image, args=())
thread3.start()

Image(value=b'', format='jpg', height='720', width='1080')

init resource stage:
Init resource success


In [ ]:
FLAG = False
thread1.join()
thread2.join()
thread3.join()

acl resource release all resource
dvpp resource release success
acl resource release stream
acl resource release context
Reset acl device  0
Release acl resource success
Close camera 0
